# camera calibration


In [20]:
import cv2
import numpy as np
import glob

# Define the dimensions of checkerboard
CHECKERBOARD = (7,7)

# Prepare object points
objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1],3), np.float32)
objp[:,:2] = np.mgrid[0:CHECKERBOARD[1],0:CHECKERBOARD[0]].T.reshape(-1,2)

objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('calibration_images/checkerboard_test.png')

for fname in images:
  img = cv2.imread(fname)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

  if ret == True:
      objpoints.append(objp)
      imgpoints.append(corners)
      cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
      cv2.imshow('img', img)
      cv2.waitKey(500)

cv2.destroyAllWindows()

# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# object detection using yolov8m

In [22]:
from ultralytics import YOLO
import cv2

def rescaleFrame(frame, scale=0.75):
    #image, video, live capture
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    return cv2.resize(frame, (width, height),interpolation=cv2.INTER_AREA)

# Load YOLO v8 model (use 'yolov8n.pt' as an example; choose appropriate variant)
model = YOLO('best.pt')

# Read the image
image = cv2.imread("img/smbox.jpeg")

# capture = cv2.VideoCapture(0)
# while True:
#     isTrue, image = capture.read()
#     # cv.imshow('video', frame)

#     if cv2.waitKey(20) & 0xFF==ord('d'):
#         break

image = rescaleFrame(image,0.5)

# Perform inference
results = model(image)

# cv2.imshow("detected",results)

# Parse results
detections = results[0].boxes  # Assuming single image input

for box in detections:
  cls = int(box.cls)
  conf = box.conf
  xyxy = box.xyxy  # (x1, y1, x2, y2)
  print(f"Class: {cls}, Confidence: {conf}, Box: {xyxy}")


0: 512x640 1 Cardboard, 95.5ms
Speed: 4.0ms preprocess, 95.5ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)
Class: 0, Confidence: tensor([0.8220]), Box: tensor([[ 40.2158,  28.3944, 609.4910, 422.4665]])


# Rotated Bounding Box Adjustment

In [23]:
import cv2
import numpy as np

def enhance_image(roi):
  """
  Enhanced image preprocessing with better contrast and noise reduction
  """
  lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
  l, a, b = cv2.split(lab)
  clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
  cl = clahe.apply(l)
  enhanced = cv2.merge([cl, a, b])
  enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)
  return enhanced

def get_binary_mask(roi):
  """
  Create a binary mask using multiple thresholding techniques
  """
  enhanced = enhance_image(roi)
  gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, (5, 5), 0)
  _, thresh1 = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  thresh2 = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                cv2.THRESH_BINARY, 11, 2)
  binary = cv2.bitwise_or(thresh1, thresh2)
  kernel = np.ones((3,3), np.uint8)
  binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
  binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
  return binary

def get_accurate_rotated_bbox(image, bbox, min_area_ratio=0.1):
  """
  Get accurate rotated bounding box with improved contour detection
  """
  x1, y1, x2, y2 = map(int, bbox)
  roi = image[y1:y2, x1:x2]
  
  roi_height, roi_width = roi.shape[:2]
  roi_area = roi_height * roi_width
  
  binary = get_binary_mask(roi)
  contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  
  if not contours:
      return None, None
  
  # Filter contours based on area
  valid_contours = []
  for contour in contours:
      area = cv2.contourArea(contour)
      if area > roi_area * min_area_ratio:
          valid_contours.append(contour)
  
  if not valid_contours:
      return None, None
  
  c = max(valid_contours, key=cv2.contourArea)
  epsilon = 0.02 * cv2.arcLength(c, True)
  approx = cv2.approxPolyDP(c, epsilon, True)
  hull = cv2.convexHull(approx)
  
  rotated_rect = cv2.minAreaRect(hull)
  box = cv2.boxPoints(rotated_rect)
  box = np.int0(box)
  
  box[:,0] += x1
  box[:,1] += y1
  
  center_x, center_y = rotated_rect[0]
  adjusted_rect = ((center_x + x1, center_y + y1), rotated_rect[1], rotated_rect[2])
  
  return adjusted_rect, box

def draw_rotated_bbox(image, rotated_box, color=(0, 255, 0), thickness=2):
  """
  Draw rotated bounding box with additional information
  """
  cv2.polylines(image, [rotated_box], True, color, thickness)
  center = np.mean(rotated_box, axis=0).astype(int)
  cv2.circle(image, tuple(center), 3, (0, 0, 255), -1)
  return image

# Main processing function
def process_detections(image, detections):
  """
  Process image with all detections
  """
  processed_image = image.copy()
  
  for box in detections:
      # Convert tensor values to Python scalars
      cls = int(box.cls.item())  # Convert class tensor to int
      conf = float(box.conf.item())  # Convert confidence tensor to float
      xyxy = [float(x) for x in box.xyxy[0]]  # Convert bbox coordinates to float
      
      rotated_rect, rotated_box = get_accurate_rotated_bbox(image, xyxy)
      
      if rotated_box is not None:
          # Draw the rotated bounding box
          processed_image = draw_rotated_bbox(processed_image, rotated_box)
          
          # Get angle and dimensions
          center, (width, height), angle = rotated_rect
          
          # Add text annotations
          text = f"Class: {cls}, Conf: {conf:.2f}"
          cv2.putText(processed_image, text, 
                     (int(center[0]), int(center[1])), 
                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
          
          print(f"Object {cls}: Center: {center}, Width: {width:.1f}, Height: {height:.1f}, Angle: {angle:.1f}")
  
  return processed_image

# Usage example:
# Assuming 'image' is your input image and 'detections' contains YOLO detections
# image = cv2.imread('your_image.jpg')
image_processed = process_detections(image, detections)
cv2.imshow('Improved Rotated Bounding Boxes', image_processed)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\ambbu\AppData\Local\Temp\ipykernel_25608\4069726375.py:65: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


Object 0: Center: (324.0, 224.5), Width: 393.0, Height: 568.0, Angle: 90.0
